In [1]:
# Logging setup
import logging
defaultLogger = logging.getLogger('default')
defaultLogger.setLevel(logging.DEBUG)

# Logging examples
# defaultLogger.debug("default debug message")
# defaultLogger.info("default info message")
# defaultLogger.warning("default warning message")
# defaultLogger.error("default error message")
# defaultLogger.critical("default critical message")

In [2]:
# Read data
from pyspark import sql

spark = (
    sql.SparkSession.builder
        .appName("Data Contract Test")
        .getOrCreate()

)

path = 'examples/spark/data/demographics.csv' #'abfss://covid@staadpstaginglakefrey.dfs.core.windows.net/raw/demographics.csv'
schema = spark.read \
            .format("csv") \
            .option("header", True) \
            .option("inferSchema", True) \
            .load(path) \
            .limit(10) \
            .schema
df = spark.read \
            .format("csv") \
            .option("header", True) \
            .schema(schema) \
            .load(path) 

defaultLogger.debug(f"Schema: {df.schema}")

In [3]:
from datacontract.data_contract import DataContract

dataset_name = "demographics"

# Create a view
df.createOrReplaceTempView(dataset_name)

# Specify data contract
data_contract_str = f"""
dataContractSpecification: 0.9.2
id: adp:datacontract:covid:demographics
info:
  title: Demographics
  version: 1.0.0
  description: Information related to the population demographics for each region.    
  owner: Google
  contact:
    name: Freya
servers:
  production:
    type: databricks
    host: {path}
    catalog: none
    schema: none
models:
  {dataset_name}: # corresponds to a table
    type: table
    fields: 
      location_key:
        type: string
        description: Unique string identifying the region
        required: true
        unique: true
      population:
        type: integer
        description: Total count of humans
        required: true
        minimum: 0
      population_male:
        type: integer
        description: Total count of males
        required: true
        minimum: 0
      population_female:
        type: integer
        description: Total count of females
        required: true
        minimum: 0
      human_development_index:
        type: double
        description: Composite index of life expectancy, education, and per capita income indicators
        required: true
        minimum: 0
        maximum: 1
"""

data_contract = DataContract(
  data_contract_str=data_contract_str, 
  spark=spark)

In [4]:
# Test data contract
run = data_contract.test()
defaultLogger.info(f"Run result: {run.result}")

INFO:default:Run result: failed


In [5]:
from rich import box
from rich import print
from rich.table import Table

def _handle_result(run):
    if run.result == "passed":
        print(
            f"🟢 data contract is valid. Run {len(run.checks)} checks. Took {(run.timestampEnd - run.timestampStart).total_seconds()} seconds.")
    else:
        print(f"🔴 data contract is invalid. Run {len(run.checks)} checks. Took {(run.timestampEnd - run.timestampStart).total_seconds()} seconds.")
    _print_table(run)

def with_markup(result):
    if result == "passed":
        return "[green]passed[/green]"
    if result == "warning":
        return "[yellow]warning[/yellow]"
    if result == "failed":
        return "[red]failed[/red]"
    if result == "error":
        return "[red]error[/red]"
    return result

def to_field(run, check):
    models = [c.model for c in run.checks]
    if len(set(models)) > 1:
        if check.field is None:
            return check.model
        return check.model + "." + check.field
    else:
        return check.field

def _print_table(run):
    table = Table(box=box.ROUNDED)
    table.add_column("Result", no_wrap=True)
    table.add_column("Check", max_width=100)
    table.add_column("Field", max_width=32)
    table.add_column("Details", max_width=50)
    for check in run.checks:
        table.add_row(with_markup(check.result), check.name, to_field(run, check), check.reason)
    print(table)

defaultLogger.info(_handle_result(run))
defaultLogger.debug(run)

🔴 data contract is invalid. Run 16 checks. Took 4.496395 seconds.

╭────────┬────────────────────────────────────────────────────────────────────┬─────────────────────────┬─────────╮
│ Result │ Check                                                              │ Field                   │ Details │
├────────┼────────────────────────────────────────────────────────────────────┼─────────────────────────┼─────────┤
│ passed │ Check that field location_key is present                           │                         │         │
│ passed │ Check that field location_key has type string                      │                         │         │
│ passed │ Check that field population is present                             │                         │         │
│ passed │ Check that field population has type integer                       │                         │         │
│ passed │ Check that field population_male is present                        │                         │         │
│ passed │ Check that field population_male has type integer                  │                         │         │
│ passed │ Check that field population_female is present                      │                         │         │
│ passed │ Check that field population_female has type integer                │                         │         │
│ passed │ Check that field human_development_index is present                │                         │         │
│ passed │ Check that field human_development_index has type double           │                         │         │
│ passed │ Check that required field location_key has no null values          │ location_key            │         │
│ passed │ Check that unique field location_key has no duplicate values       │ location_key            │         │
│ failed │ Check that required field population has no null values            │ population              │         │
│ failed │ Check that required field population_male has no null values       │ population_male         │         │
│ failed │ Check that required field population_female has no null values     │ population_female       │         │
│ failed │ Check that required field human_development_index has no null      │ human_development_index │         │
│        │ values                                                             │                         │         │
╰────────┴────────────────────────────────────────────────────────────────────┴─────────────────────────┴─────────╯

INFO:default:None
DEBUG:default:runId=UUID('f383ad09-236e-4138-bbe2-0118c3492291') dataContractId='adp:datacontract:covid:demographics' dataContractVersion='1.0.0' dataProductId=None outputPortId=None server='production' timestampStart=datetime.datetime(2024, 3, 19, 13, 11, 12, 365734, tzinfo=datetime.timezone.utc) timestampEnd=datetime.datetime(2024, 3, 19, 13, 11, 16, 862129, tzinfo=datetime.timezone.utc) result='failed' checks=[Check(type='schema', name='Check that field location_key is present', result='passed', engine='soda-core', reason='', model='demographics', field=None, details=None), Check(type='schema', name='Check that field location_key has type string', result='passed', engine='soda-core', reason='', model='demographics', field=None, details=None), Check(type='schema', name='Check that field population is present', result='passed', engine='soda-core', reason='', model='demographics', field=None, details=None), Check(type='schema', name='Check that field population has ty

In [6]:
# See failed checks
failed_checks = [check for check in run.checks if check.result == 'failed']
defaultLogger.info(f"Failed checks: {failed_checks}")

INFO:default:Failed checks: [Check(type='schema', name='Check that required field population has no null values', result='failed', engine='soda-core', reason='', model='demographics', field='population', details=None), Check(type='schema', name='Check that required field population_male has no null values', result='failed', engine='soda-core', reason='', model='demographics', field='population_male', details=None), Check(type='schema', name='Check that required field population_female has no null values', result='failed', engine='soda-core', reason='', model='demographics', field='population_female', details=None), Check(type='schema', name='Check that required field human_development_index has no null values', result='failed', engine='soda-core', reason='', model='demographics', field='human_development_index', details=None)]


In [7]:
# Fail if failed checks
if not run.has_passed():
    raise Exception("Data contract validation failed")

Exception: Data contract validation failed